In [1]:
!pip install transformers
!pip install thefuzz

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\anton\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\anton\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


- https://en.wikipedia.org/wiki/Judo

- https://en.wikipedia.org/wiki/List_of_judo_techniques

- https://en.wikipedia.org/wiki/List_of_judoka

- https://martialarts.fandom.com/wiki/Judo

- https://chas-ma.com/JudoManual/Chapter_2%28HistoryofJudo%29.pdf

- https://www.ijf.org/history

- https://blackbelttrek.com/judo-vs-jiu-jitsu-the-ultimate-comparison/

In [ ]:
import json
import os
from transformers import pipeline
from thefuzz import fuzz

# Lade das vortrainierte Modell
model_name = "deepset/roberta-base-squad2"
nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)

data_dir = "data-sources"
context_file = "complete_context.txt"

# Lese alle .txt-Dateien im Verzeichnis data-sources ein
context_parts = []
for filename in os.listdir(data_dir):
    if filename.endswith(".txt"):
        file_path = os.path.join(data_dir, filename)
        with open(file_path, "r", encoding="utf-8") as f:
            context_parts.append(f.read())

# Verbinde alle Texte zu einem Gesamttext
context = "\n".join(context_parts)

# Speichere den gesamten Kontext in eine Datei
with open(context_file, "w", encoding="utf-8") as f:
    f.write(context)

print('Context length: ', len(context))

Device set to use cpu


Context length:  147981


In [ ]:
# Lade die Fragen
with open("q_100.json", "r", encoding="utf-8") as f:
    questions_data = json.load(f)

THRESHOLD = 60
correct = 0
total = len(questions_data)

# Teste alle Fragen
for item in questions_data:
    question = item["question"]
    expected_answer = item["answer"]

    # Frage an das Modell stellen
    QA_input = {
        'question': question,
        'context': context
    }
    result = nlp(QA_input)
    predicted_answer = result["answer"]

    # Berechne die Ähnlichkeit der Antworten
    similarity = fuzz.ratio(predicted_answer.lower(), expected_answer.lower())

    # Vergleiche mit Schwellenwert
    if similarity >= THRESHOLD:
        correct += 1
        print(f"✅ Frage: {question}")
        print(f"   Erwartet: {expected_answer}")
        print(f"   Erhalten: {predicted_answer}")
        print(f"   Ähnlichkeit: {similarity}%\n")
    else:
        print(f"❌ Frage: {question}")
        print(f"   Erwartet: {expected_answer}")
        print(f"   Erhalten: {predicted_answer}")
        print(f"   Ähnlichkeit: {similarity}%\n")

# Berechne die Erfolgsrate
accuracy = (correct / total) * 100
print(f"\n✅ Erfolgsrate: {accuracy:.2f}% ({correct}/{total} korrekt)")


Device set to use cpu


Context length:  6864
✅ Frage: Who created Judo?
   Erwartet: Kanō Jigorō
   Erhalten: Kanō Jigorō
   Ähnlichkeit: 100%

✅ Frage: In what year was Judo founded?
   Erwartet: 1882
   Erhalten: 1882
   Ähnlichkeit: 100%

✅ Frage: Where was the Kodokan Judo Institute established?
   Erwartet: Tokyo
   Erhalten: Tokyo
   Ähnlichkeit: 100%

✅ Frage: What does 'Judo' mean?
   Erwartet: Gentle way
   Erhalten: Gentle way
   Ähnlichkeit: 100%

✅ Frage: Which martial art did Judo originate from?
   Erwartet: Jujutsu
   Erhalten: Jujutsu
   Ähnlichkeit: 100%

✅ Frage: What are the two main principles of Judo?
   Erwartet: Seiryoku-Zenyo and Jita-Kyoei
   Erhalten: Seiryoku-Zenyo and Jita-Kyoei
   Ähnlichkeit: 100%

❌ Frage: What are the three main categories of Judo techniques?
   Erwartet: Nage-waza, katame-waza, atemi-waza
   Erhalten: Ground techniques
   Ähnlichkeit: 20%

✅ Frage: What is the name of the uniform worn in Judo?
   Erwartet: Judogi
   Erhalten: Judogi
   Ähnlichkeit: 100%

✅ Fr